In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("Json to DataFrame") \
        .getOrCreate()

df = spark.read.json("gs://dataproc-staging-us-east1-766635705898-g9ye7adw/final_project/data_feature.json_part-00000-81821db2-daa5-40c6-8ebe-5eaa98c07e5c-c000.json")
df.show()

23/12/02 22:36:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----------+------------------+------------------+--------------+-----------+--------+--------+--------+-------------------+
|is_helpful|          scoreVec|sum_compound_score|summary_length|text_length|text_neg|text_neu|text_pos|weekdayOrWeekendVec|
+----------+------------------+------------------+--------------+-----------+--------+--------+--------+-------------------+
|         0|    {[], 5, 0, []}|               0.0|            29|        854|   0.259|   0.664|   0.077| {[0], 1, 0, [1.0]}|
|         0|{[1], 5, 0, [1.0]}|               0.0|            23|        312|   0.283|   0.681|   0.036| {[0], 1, 0, [1.0]}|
|         1|    {[], 5, 0, []}|               0.0|            29|       1709|   0.017|   0.876|   0.107|     {[], 1, 0, []}|
|         1|    {[], 5, 0, []}|               0.0|            54|       1072|   0.005|   0.796|   0.199| {[0], 1, 0, [1.0]}|
|         1|    {[], 5, 0, []}|            0.3612|            42|        725|   0.018|   0.858|   0.124| {[0], 1, 0, [1.0]}|


In [2]:
df.schema

StructType([StructField('is_helpful', LongType(), True), StructField('scoreVec', StructType([StructField('indices', ArrayType(LongType(), True), True), StructField('size', LongType(), True), StructField('type', LongType(), True), StructField('values', ArrayType(DoubleType(), True), True)]), True), StructField('sum_compound_score', DoubleType(), True), StructField('summary_length', LongType(), True), StructField('text_length', LongType(), True), StructField('text_neg', DoubleType(), True), StructField('text_neu', DoubleType(), True), StructField('text_pos', DoubleType(), True), StructField('weekdayOrWeekendVec', StructType([StructField('indices', ArrayType(LongType(), True), True), StructField('size', LongType(), True), StructField('type', LongType(), True), StructField('values', ArrayType(DoubleType(), True), True)]), True)])

In [3]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

# List of input columns for features (excluding the target variable and nested columns)
input_cols = ['sum_compound_score', 'summary_length', 'text_length', 'text_neg', 'text_neu', 'text_pos']

# Assemble the features into a single feature vector
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")
df = assembler.transform(df)

# Select only the features and the target column
df = df.select("features", df.is_helpful.alias("label"))
train_data, test_data = df.randomSplit([0.75, 0.25], seed=42)
from pyspark.ml.classification import LogisticRegression

# Initialize the Logistic Regression model
lr = LogisticRegression()

# Train the model
model = lr.fit(train_data)


In [4]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Make predictions on the test data
predictions = model.transform(test_data)

# Initialize the evaluator
evaluator = BinaryClassificationEvaluator()

# Evaluate the model
accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

print(f"Test Area Under ROC: {accuracy}")
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Use the MulticlassClassificationEvaluator to evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

# Calculate accuracy on the test data
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy}")


Test Area Under ROC: 0.7001703367078497
Test Accuracy: 0.6862546437014522


In [5]:
from pyspark.ml.classification import RandomForestClassifier

# Initialize the Random Forest model
rf = RandomForestClassifier()

# Train the model on the training data
rf_model = rf.fit(train_data)

# Make predictions on the test data
rf_predictions = rf_model.transform(test_data)

# Evaluate the model using MulticlassClassificationEvaluator for accuracy
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(rf_predictions)
print(f"Test Accuracy: {accuracy}")

# Evaluate the model using BinaryClassificationEvaluator for area under ROC
binary_evaluator = BinaryClassificationEvaluator()
roc_auc = binary_evaluator.evaluate(rf_predictions, {binary_evaluator.metricName: "areaUnderROC"})
print(f"Test Area Under ROC: {roc_auc}")


Test Accuracy: 0.70145221209051
Test Area Under ROC: 0.7231770185453665


In [10]:
pip install sparkxgb

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 MB 5.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 25.2 MB/s eta 0:00:00
  Created wheel for sparkxgb: filename=sparkxgb-0.1-py3-none-any.whl size=5627 sha256=43dd086c594fd4c4c08fdeec8f566084dc472d27c6e293959ea5c31e1d7b696c
  Stored in directory: /root/.cache/pip/wheels/b7/0c/a1/786408e13056fabeb8a72134e101b1e142fc95905c7b0e2a71
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767582 sha256=9e1c95ef2d5a52d5858cf366a934f9110e643693d17885b3639daba306b0ac1e
  Stored in directory: /root/.cache/pip/wheels/a0/3f/72/8efd988f9ae041f051c75e6834cd92dd6d13a726e206e8b6f3
Successfully built sparkxgb pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5
  Attempting uninstall: pyspark


In [7]:
from pyspark.ml.classification import LinearSVC

# Initialize the LinearSVC model
svm = LinearSVC()

# Train the model on the training data
svm_model = svm.fit(train_data)
# Make predictions on the test data
svm_predictions = svm_model.transform(test_data)

# Evaluate the model using MulticlassClassificationEvaluator for accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(svm_predictions)
print(f"Test Accuracy: {accuracy}")


Test Accuracy: 0.6786555942094041
